## Data Preparation

You should prepare the following things before running this step. Please refer to the `example_data` folder for guidance:

1. **NIfTI images** of head CT resampled to a voxel size of **[1,1,2.5] mm³**.  
   - Please refer to ```example_data/data/raw_data``` for an example

---

## Simulate motion-corrupted image and its partial angle reconstruction (PAR) for supervised training

In this script, we apply simulated motion to a motion-free head CT scan and therefore record the motion parameters (used as ground truth in model training) and generate motion-corrupted images as well as partial angle reconstrutions (used as model input).
- motion parameter and image example: ```example_data/data/simulated_data_3D_spline_6degrees```.
    
- PAR example: ```example_data/data/PAR_3D_spline_6degrees```

---

### Docker environment
1. Please use `docker/docker_tensorflow`, it will build a tensorflow docker


In [1]:
# %%
import os, sys
sys.path.append('/workspace/Documents')

import numpy as np
import cupy as cp
import glob as gb
import nibabel as nb
import math
import pandas as pd
import os
from skimage.measure import block_reduce

import HeadCT_MotionCorrection_PARDL.motion_simulator.motion_simulation.ct_basic as ct
import HeadCT_MotionCorrection_PARDL.functions_collection as ff
import HeadCT_MotionCorrection_PARDL.motion_simulator.transformation as transform
import HeadCT_MotionCorrection_PARDL.Data_processing as dp
import ct_projector.projector.cupy as ct_projector

main_path = '/mnt/camca_NAS/motion_correction/'  # replace with your main path

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### define default parameters

In [2]:
# define motion range:
motion_type = '3D_spline_6degrees'
save_folder_img = os.path.join(main_path,'example_data/data','simulated_data_'+motion_type)
save_folder_PAR = os.path.join(main_path,'example_data/data','PAR_'+motion_type)
ff.make_folder([save_folder_img, save_folder_PAR])

motion_dim = 3
amplitude_max = 5
displacement_max = 2.5
change_direction_limit = 2
CP_num = 5

geometry = 'fan'
total_view = 1400  ### 2340 views by default
gantry_rotation_time = 500 #unit ms, 500ms by default
view_increment = 28 # increment in gantry views

### define patient list

In [3]:
# define the patient list
data_folder = os.path.join(main_path,'example_data/data/raw_data/')
patient_list= ff.find_all_target_files(['*/*'],data_folder)
print(len(patient_list))

1


### motion simulation

In [4]:
for p in patient_list:
    patient_subid = os.path.basename(p)
    patient_id = os.path.basename(os.path.dirname(p))

    print('patient: ',patient_id, patient_subid)
    
    save_folder_img_patient = os.path.join(save_folder_img,patient_id,patient_subid)
    ff.make_folder([os.path.dirname(save_folder_img_patient), save_folder_img_patient])

    img_file = os.path.join(p,'img_2.5mm.nii.gz')

    img,spacing,img_affine = ct.basic_image_processing(img_file)

    spacing = [2.5, 1.0, 1.0]
    print('nib image shape: ',img.shape, ' spacing: ',spacing)

    # define projectors
    img = img[np.newaxis, ...]
    projector = ct.define_forward_projector(img,spacing,total_view)
    fbp_projector = ct.backprojector(img,spacing)


    # very important - make sure that the arrays are saved in C order
    cp.cuda.Device(0).use()
    ct_projector.set_device(0)



    # do simulation
    L = np.arange(1,2)

    for random_i in L:

        t = np.linspace(0, gantry_rotation_time, CP_num, endpoint=True)

        # create folder
        if random_i == 0:
            random_folder = os.path.join(save_folder_img_patient,'static')
        else:
            random_folder = os.path.join(save_folder_img_patient,'random_'+str(random_i))
        ff.make_folder([random_folder, os.path.join(random_folder,'image_data')])

        print('\n',random_i  , 'random')


        # use previous motion if available
        saved_motion_parameter_file = os.path.join(random_folder,  'motion_parameters.npy')

        if os.path.isfile(saved_motion_parameter_file) == 1:
            previous_motion_parameter = np.load(saved_motion_parameter_file, allow_pickle = True)
            amplitude_tx_mm = previous_motion_parameter[0,:][0]
            amplitude_ty_mm = previous_motion_parameter[1,:][0]
            amplitude_tz_mm = previous_motion_parameter[2,:][0]
            amplitude_rx_degree = previous_motion_parameter[3,:][0]
            amplitude_ry_degree = previous_motion_parameter[4,:][0]
            amplitude_rz_degree = previous_motion_parameter[5,:][0]
            sga = previous_motion_parameter[6,:][0]

        else:
            ValueError('no previous motion saved')
            while True:
                amplitude_tx_mm = transform.motion_control_point_generation(1, CP_num, amplitude_max, displacement_max, change_direction_limit, print_result = False)[:,0]
                amplitude_ty_mm = transform.motion_control_point_generation(1, CP_num, amplitude_max, displacement_max, change_direction_limit, print_result = False)[:,0]
                amplitude_tz_mm = transform.motion_control_point_generation(1, CP_num, amplitude_max, displacement_max, change_direction_limit, print_result = False)[:,0]
                amplitude_rx_degree = transform.motion_control_point_generation(1, CP_num, amplitude_max, displacement_max, change_direction_limit, print_result = False)[:,0]
                amplitude_ry_degree = transform.motion_control_point_generation(1, CP_num, amplitude_max, displacement_max, change_direction_limit, print_result = False)[:,0]
                amplitude_rz_degree = transform.motion_control_point_generation(1, CP_num, amplitude_max, displacement_max, change_direction_limit, print_result = False)[:,0]
                if np.max(abs(amplitude_rx_degree))+ np.max(abs(amplitude_ry_degree)) <= 7:
                    print('rx+ry: ', np.max(abs(amplitude_rx_degree))+ np.max(abs(amplitude_ry_degree)))
                    break

            sga = int(np.random.uniform(0,90)) # starting gantry angle
            if random_i == 0:
                amplitude_tx_mm = [0, 0, 0, 0, 0]
                amplitude_ty_mm = [0, 0, 0, 0, 0]
                amplitude_tz_mm = [0, 0, 0, 0, 0]
                amplitude_rx_degree = [0, 0, 0, 0, 0]
                amplitude_ry_degree = [0, 0, 0, 0, 0]
                amplitude_rz_degree = [0, 0, 0, 0, 0]

            # save motion parameters
            parameter_file = os.path.join(random_folder,'motion_parameters.txt')
            ff.txt_writer(parameter_file,False,[t.tolist(),amplitude_tx_mm, amplitude_ty_mm, amplitude_tz_mm, amplitude_rx_degree, amplitude_ry_degree, amplitude_rz_degree, [sga],[total_view],[gantry_rotation_time]],['time_points','translation_x_CP','translation_y_CP','translation_z_CP', 'rotation_x_CP', 'rotation_y_CP','rotation_z_CP','starting_gantry_angle', 'total_projection_view','gantry_rotation_time(ms)'])

            parameter_file = os.path.join(random_folder,'motion_parameters.npy')
            np.save(parameter_file, np.array([[amplitude_tx_mm],[amplitude_ty_mm],[amplitude_tz_mm],[amplitude_rx_degree],[amplitude_ry_degree],[amplitude_rz_degree],[sga], [t], [total_view], [gantry_rotation_time]], dtype=object))



        # prepare spline fit
        spline_tx = transform.interp_func(t, np.asarray([i/spacing[1] for i in amplitude_tx_mm]))
        spline_ty = transform.interp_func(t, np.asarray([i/spacing[2] for i in amplitude_ty_mm]))
        spline_tz = transform.interp_func(t, np.asarray([i/spacing[0] for i in amplitude_tz_mm]))
        spline_rx = transform.interp_func(t,np.asarray([i / 180 * np.pi for i in amplitude_rx_degree]))
        spline_ry = transform.interp_func(t,np.asarray([i / 180 * np.pi for i in amplitude_ry_degree]))
        spline_rz = transform.interp_func(t,np.asarray([i / 180 * np.pi for i in amplitude_rz_degree]))
        angles = ff.get_angles_zc(total_view, 360 ,sga)

        # generate forward projection
        projection = ct.fp_w_spline_motion_model(img, projector, angles, spline_tx, spline_ty, spline_tz, spline_rx, spline_ry, spline_rz, geometry, total_view = total_view, gantry_rotation_time = gantry_rotation_time, slice_num = None, increment = view_increment, order = 3)

        # # save fp
        projection_save_version = nb.Nifti1Image(projection[:,:,0,:], img_affine)
        nb.save(projection_save_version, os.path.join(random_folder,'projection.nii.gz'))
        # projection = nb.load(os.path.join(random_folder,'projection.nii.gz')).get_fdata()
        # projection = np.expand_dims(projection, axis=2)
        # print('projection shape: ', projection.shape) 

        # # generate backprojection
        recon = ct.filtered_backporjection(projection,angles,projector,fbp_projector, geometry, back_to_original_value=True)

        # save recon
        recon_nb_image = np.rollaxis(recon,0,3) 
        print(recon_nb_image.shape)
        nb.save(nb.Nifti1Image(recon_nb_image,img_affine), os.path.join(random_folder,'image_data','recon.nii.gz'))
        # only pick 50 slices
        nb.save(nb.Nifti1Image(recon_nb_image[:,:,10:60],img_affine), os.path.join(random_folder,'image_data','recon_partial.nii.gz'))

        # make PAR
        if random_i == 0:
            print('static, no need to make PAR')
            continue

        K = 12
        save_folder2 = os.path.join(save_folder_PAR,patient_id, patient_subid, 'random_' + str(random_i),'slice_0_to_50')
        ff.make_folder([os.path.join(save_folder_PAR,patient_id), os.path.join(save_folder_PAR,patient_id, patient_subid), os.path.join(save_folder_PAR,patient_id, patient_subid, 'random_' + str(random_i)), save_folder2])
        
        sinogram_segments, center_angle_index, num_angles_in_one_segment, segment_indexes = ct.divide_sinogram_new(projection, K , total_view)

        PAR_collections = ct.make_PAR_new(sinogram_segments, segment_indexes, angles, img[0,...].shape, projector, fbp_projector, 'fan')

        PAR_collections = np.rollaxis(PAR_collections,1,4)
        # only pick 50 slices
        PAR_collections = PAR_collections[:,:,:,10:60]
        print(PAR_collections.shape)

        PAR_collections_ds = block_reduce(PAR_collections, block_size=(1,2,2,1), func=np.mean)
        print(PAR_collections_ds.shape)

            
        crop_img = np.zeros([2*K + 1, 128,128 , PAR_collections_ds.shape[-1]])

        for j in range(0,PAR_collections_ds.shape[0]):
            crop_img[j,...] = dp.crop_or_pad(PAR_collections_ds[j,...], [128,128,PAR_collections_ds.shape[-1] ], np.min(PAR_collections_ds[j,...]))

        print('crop final image shape: ' ,crop_img.shape)

        nb.save(nb.Nifti1Image(crop_img, img_affine), os.path.join(save_folder2,'PARs_ds_crop.nii.gz'))

patient:  MO101701M000001 MO001A000001
nib image shape:  (66, 220, 220)  spacing:  [2.5, 1.0, 1.0]

 1 random


/workspace/Documents/HeadCT_MotionCorrection_PARDL/motion_simulator/transformation/rotation_matrix_from_angle.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(
/workspace/Documents/HeadCT_MotionCorrection_PARDL/motion_simulator/transformation/rotation_matrix_from_angle.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(
/workspace/Documents/HeadCT_MotionCorrection_PARDL/motion_simulator/transformation/rotation_matrix_from_angle.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (whic

(220, 220, 66)
(25, 220, 220, 50)
(25, 110, 110, 50)
crop final image shape:  (25, 128, 128, 50)
